In [31]:
import pandas as pd
import json
import glob
import os
import re
from pathlib import Path
from pydracor import DraCorAPI

## 1. Loading experiments results from JSON files to a single dataframe

In [7]:
EXPERIMENT_PREFIXES = [
    "1-1",
    "1-2",
    "1-3",
    "1-4",
    "1-5",
    "3-1",
    "4-1",
    "4-2",
    "4-3",
    "4-4",
    "5-1",
    "5-2",
    "5-3",
    "5-4",
]

In [52]:
# Path to the uploaded files
path = "results/sonnet-4/extracted/*.json"

rows = []

for file in glob.glob(path):
    with open(file, "r") as f:
        data = json.load(f)

    filename = os.path.basename(file)
    
    # Experiment ID is always the first part before the first "_"
    experiment_id = filename.split("_")[0]  # e.g. "1-1"

    # Extract the `response` field (if missing, set to None)
    response = data.get("response", None)
    tool_chain = data.get("tool_chain", None)
    success = data.get("success", False)

    rows.append({
        "filename": filename,
        "experiment_id": experiment_id,
        "success": success,
        "response": response,
        "tool_chain": tool_chain,
    })

df = pd.DataFrame(rows)
df


,filename,experiment_id,success,response,tool_chain
0,4-4_12_extracted-info.json,4-4,True,"Based on the data from ItaDraCor, I can now an...","[get_corpus, get_corpus_metadata_csv]"
1,1-2_13_extracted-info.json,1-2,True,"Based on the metadata I retrieved, I can count...","[get_plays_in_corpus_by_title_helper, get_play..."
2,5-2_12_extracted-info.json,5-2,True,Emilia,[get_play_characters]
3,5-1_15_extracted-info.json,5-1,True,Marinelli,[get_play_characters]
4,1-5_11_extracted-info.json,1-5,True,14,"[get_corpora, get_plays_in_corpus_by_title_hel..."
...,...,...,...,...,...
172,1-5_20_extracted-info.json,1-5,True,14,"[get_plays_in_corpus_by_title_helper, get_play..."
173,3-2_16_extracted-info.json,3-2,False,,[]
174,4-3_4_extracted-info.json,4-3,True,"Based on the Swedish drama corpus data, here's...",[get_corpus_metadata_csv]
175,3-1_11_extracted-info.json,3-1,True,"Based on the corpora information, I need to ca...",[get_corpora]


In [53]:
df['experiment_id'].value_counts()

experiment_id
4-4    13
1-5    13
1-1    12
3-1    12
5-2    11
4-3    11
4-1    11
1-4    11
5-4    11
3-2    11
5-3    11
1-2    10
5-1    10
4-2    10
1-3    10
2-1    10
Name: count, dtype: int64

## 2. Post-processing LLM responses for better automatic evaluation:

In [54]:
def extract_last_number(s):
    if s is None:
        return None
    # find all groups of digits
    nums = re.findall(r"\d+", str(s))
    if not nums:
        return None
    return int(nums[-1])  # take the last one

In [55]:
df["numeric_response"] = df["response"].apply(extract_last_number)

In [56]:
df

,filename,experiment_id,success,response,tool_chain,numeric_response
0,4-4_12_extracted-info.json,4-4,True,"Based on the data from ItaDraCor, I can now an...","[get_corpus, get_corpus_metadata_csv]",25.0
1,1-2_13_extracted-info.json,1-2,True,"Based on the metadata I retrieved, I can count...","[get_plays_in_corpus_by_title_helper, get_play...",103.0
2,5-2_12_extracted-info.json,5-2,True,Emilia,[get_play_characters],NaN
3,5-1_15_extracted-info.json,5-1,True,Marinelli,[get_play_characters],NaN
4,1-5_11_extracted-info.json,1-5,True,14,"[get_corpora, get_plays_in_corpus_by_title_hel...",14.0
...,...,...,...,...,...,...
172,1-5_20_extracted-info.json,1-5,True,14,"[get_plays_in_corpus_by_title_helper, get_play...",14.0
173,3-2_16_extracted-info.json,3-2,False,,[],NaN
174,4-3_4_extracted-info.json,4-3,True,"Based on the Swedish drama corpus data, here's...",[get_corpus_metadata_csv],55.0
175,3-1_11_extracted-info.json,3-1,True,"Based on the corpora information, I need to ca...",[get_corpora],39.0


In [57]:
def extract_all_numbers(s):
    if s is None:
        return []
    # find all groups of digits
    nums = re.findall(r"\d+", str(s))
    return [int(n) for n in nums]  # convert to ints

df["all_numbers"] = df["response"].apply(extract_all_numbers)

In [58]:
df[df['experiment_id']=='1-5'][['filename', 'response', 'numeric_response', 'all_numbers']]

,filename,response,numeric_response,all_numbers
4,1-5_11_extracted-info.json,14,14.0,[14]
15,1-5_13_extracted-info.json,14,14.0,[14]
17,1-5_4_extracted-info.json,"Perfect! I can count the characters in ""Der No...",14.0,"[14, 14]"
22,1-5_12_extracted-info.json,"Perfect! I can see that ""Der Nollhart"" has 14 ...",14.0,"[14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
31,1-5_14_extracted-info.json,14,14.0,[14]
34,1-5_3_extracted-info.json,14,14.0,[14]
41,1-5_15_extracted-info.json,"Perfect! I can see from the metadata that ""Der...",14.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
56,1-5_1_extracted-info.json,"Perfect! I can count the characters in ""Der No...",14.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
58,1-5_16_extracted-info.json,14,14.0,[14]
67,1-5_17_extracted-info.json,14,14.0,[14]


In [59]:
df[df['experiment_id']=='1-5'][['response', 'numeric_response', 'all_numbers']]

,response,numeric_response,all_numbers
4,14,14.0,[14]
15,14,14.0,[14]
17,"Perfect! I can count the characters in ""Der No...",14.0,"[14, 14]"
22,"Perfect! I can see that ""Der Nollhart"" has 14 ...",14.0,"[14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
31,14,14.0,[14]
34,14,14.0,[14]
41,"Perfect! I can see from the metadata that ""Der...",14.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
56,"Perfect! I can count the characters in ""Der No...",14.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
58,14,14.0,[14]
67,14,14.0,[14]


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   filename          177 non-null    object 
 1   experiment_id     177 non-null    object 
 2   success           177 non-null    bool   
 3   response          177 non-null    object 
 4   tool_chain        177 non-null    object 
 5   numeric_response  111 non-null    float64
 6   all_numbers       177 non-null    object 
dtypes: bool(1), float64(1), object(5)
memory usage: 8.6+ KB


In [62]:
# stats = (
#     df_filtered.groupby("experiment_id")["numeric_response"]
#       .agg(["count", "mean", "std", "var", "min", "max"])
# )

# # add range as max-min
# stats["range"] = stats["max"] - stats["min"]

# stats

In [63]:
df.groupby("experiment_id").size()

experiment_id
1-1    12
1-2    10
1-3    10
1-4    11
1-5    13
2-1    10
3-1    12
3-2    11
4-1    11
4-2    10
4-3    11
4-4    13
5-1    10
5-2    11
5-3    11
5-4    11
dtype: int64

In [64]:
df.groupby("experiment_id")["numeric_response"].std()

experiment_id
1-1    3.812778e+01
1-2    2.366667e+01
1-3    3.522562e+01
1-4    5.334280e+00
1-5    0.000000e+00
2-1             NaN
3-1    1.549516e+01
3-2             NaN
4-1    0.000000e+00
4-2    9.600307e+02
4-3    9.688247e+02
4-4    4.741526e+02
5-1             NaN
5-2             NaN
5-3             NaN
5-4    1.745211e+14
Name: numeric_response, dtype: float64

### Normalise responses to select-the-corpus questions (3-1, 3-2)

In [65]:
# normalised response will contain the same as numeric_response for numeric questions 
# but also corpus slugs for 'which corpus' questions
df['normalised_response'] = df['numeric_response'].astype(object)

In [66]:
# this should all be replaced by the corpus slugs 
df[df['experiment_id'].isin(['3-1', '3-2'])]['normalised_response']


12      88.0
47     424.0
62       NaN
133     39.0
136     39.0
137      NaN
141      NaN
144     39.0
146     39.0
153      NaN
155      NaN
156     39.0
157     39.0
159      NaN
161     39.0
162      NaN
164      NaN
166      NaN
167     39.0
169      NaN
171      NaN
173      NaN
175     39.0
Name: normalised_response, dtype: object

In [67]:
crpra = DraCorAPI().get_corpora()

In [68]:
slugs = [corpus.name for corpus in crpra]

In [69]:
_pattern = re.compile(r'\b(?:' + '|'.join(slugs) + r')\b', flags=re.IGNORECASE)

def find_last_corpus_slug(text: str) -> str | None:
    """Return the last DraCor slug mentioned as a whole word, or None."""
    last = None
    for match in _pattern.finditer(text):
        last = match.group(0).lower()  # normalize to lowercase slug
    return last

In [70]:
mask = df['experiment_id'].isin(['3-1', '3-2'])
df.loc[mask, 'normalised_response'] = df.loc[mask, 'response'].apply(find_last_corpus_slug)

In [71]:
df[df['experiment_id'].isin(['3-1', '3-2'])][['success','response','normalised_response']]

,success,response,normalised_response
12,True,"Based on the corpus data, I need to calculate ...",gersh
47,True,Based on the data I've collected from multiple...,ger
62,False,,None
133,True,"Looking at the data, I need to calculate the m...",gersh
136,True,"Based on the corpus data, I need to calculate ...",gersh
137,False,,None
141,False,,None
144,True,"Looking at the corpus data, I need to calculat...",gersh
146,True,"Looking at the corpora data, I need to calcula...",gersh
153,True,rom,rom


In [77]:
df[(df['experiment_id'].isin(['3-1', '3-2']) & df['success']==True)][['experiment_id','success','response','normalised_response']]

,experiment_id,success,response,normalised_response
12,3-1,True,"Based on the corpus data, I need to calculate ...",gersh
47,3-2,True,Based on the data I've collected from multiple...,ger
133,3-1,True,"Looking at the data, I need to calculate the m...",gersh
136,3-1,True,"Based on the corpus data, I need to calculate ...",gersh
144,3-1,True,"Looking at the corpus data, I need to calculat...",gersh
146,3-1,True,"Looking at the corpora data, I need to calcula...",gersh
153,3-1,True,rom,rom
156,3-1,True,"Looking at the corpus data, I need to calculat...",gersh
157,3-1,True,"Based on the corpus data, I need to calculate ...",gersh
161,3-1,True,"Looking at the corpus data, I need to calculat...",gersh


In [78]:
df.to_csv("results/compiled_responses.csv", index=False)

## 3. Loading manually-defined correct responses

In [47]:
correct = pd.read_csv("preliminary_work/compiled_manual_answers.csv")

In [48]:
correct

,ID,Correct Answer
0,1-1,103
1,1-2,103
2,1-3,103
3,1-4,103
4,1-5,14
5,2-1,9.19
6,3-1,GerShDraCor
7,3-2,French
8,4-1,Open question
9,4-2,Open question


In [55]:
correct_dict = dict(zip(correct["ID"], correct["Correct Answer"]))

In [56]:
df['correct_answer'] = df['experiment_id'].map(correct_dict)

In [58]:
df.head()

,filename,experiment_id,response,tool_chain,numeric_response,all_numbers,correct_answer
0,4-4_12_extracted-info.json,4-4,"Based on the data from ItaDraCor, I can now an...","[get_corpus, get_corpus_metadata_csv]",25.0,"[10, 30, 15, 20, 15, 16, 20, 25, 17, 18, 20, 3...",Open question
1,1-2_13_extracted-info.json,1-2,"Based on the metadata I retrieved, I can count...","[get_plays_in_corpus_by_title_helper, get_play...",103.0,[103],103
2,5-2_12_extracted-info.json,5-2,Emilia,[get_play_characters],NaN,[],Emilia
3,5-1_15_extracted-info.json,5-1,Marinelli,[get_play_characters],NaN,[],Marinelli
4,1-5_11_extracted-info.json,1-5,14,"[get_corpora, get_plays_in_corpus_by_title_hel...",14.0,[14],14


In [65]:
print(df[['experiment_id', 'numeric_response', 'correct_answer']].head(10))

  experiment_id  numeric_response correct_answer
0           4-4              25.0  Open question
1           1-2             103.0            103
2           5-2               NaN         Emilia
3           5-1               NaN      Marinelli
4           1-5              14.0             14
5           1-1             103.0            103
6           5-2               NaN         Emilia
7           1-2             103.0            103
8           4-4               8.0  Open question
9           1-1              29.0            103


In [70]:
df_strictly_numeric = df[df['experiment_id'].str.startswith('1-') | 
                         df['experiment_id'].str.startswith('2-') ]

In [75]:
df_strictly_numeric.shape

(54, 7)

In [79]:
print(df_strictly_numeric[['experiment_id', 'numeric_response', 'correct_answer']].head(10))

   experiment_id  numeric_response correct_answer
1            1-2             103.0            103
4            1-5              14.0             14
5            1-1             103.0            103
7            1-2             103.0            103
9            1-1              29.0            103
14           1-1             103.0            103
15           1-5              14.0             14
17           1-5              14.0             14
19           1-2             103.0            103
21           1-5              14.0             14


In [83]:
df_strictly_numeric['correct_answer'] = df_strictly_numeric['correct_answer'].astype(int)

## 4. Evaluating correctness of the LLM response (hit & miss table)

In [90]:
# add helper columns
df_strictly_numeric = df_strictly_numeric.copy()
df_strictly_numeric["is_correct"] = (df_strictly_numeric["numeric_response"] == df_strictly_numeric["correct_answer"]).astype(int)
df_strictly_numeric["iteration"] = df_strictly_numeric.groupby("experiment_id").cumcount() + 1

# build the wide table (one column per iteration)
hit_table = (
    df_strictly_numeric.pivot(index="experiment_id", columns="iteration", values="is_correct")
      .sort_index()
      .sort_index(axis=1)
      .astype("Int64")         # keeps blanks for missing iterations
)

# prepend the summary column "X correct answers of Y total answers"
summary = (
    df_strictly_numeric.groupby("experiment_id")["is_correct"]
      .agg(["sum", "count"])
      .assign(label=lambda s: s.apply(lambda r: f"{r['sum']} correct answers of {r['count']} total answers", axis=1))
)

hit_table["Summary"] = summary.loc[hit_table.index, "label"]

# preview + save
hit_table

iteration,1,2,3,4,5,6,7,8,9,10,11,12,Summary
experiment_id,,,,,,,,,,,,,
1-1,1,0,1,0,0,0,1,0,1,1,0,1,6 correct answers of 12 total answers
1-2,1,1,1,1,1,1,1,0,1,<NA>,<NA>,<NA>,8 correct answers of 9 total answers
1-3,0,0,1,0,0,1,0,0,1,1,<NA>,<NA>,4 correct answers of 10 total answers
1-4,0,0,0,0,0,0,0,0,0,0,0,<NA>,0 correct answers of 11 total answers
1-5,1,1,1,1,1,1,1,1,1,1,1,1,12 correct answers of 12 total answers


In [91]:
hit_table.to_csv("hit_miss_table.csv")

The version with "✅" and "❌" emojis:

In [89]:
# add helper columns
df_strictly_numeric = df_strictly_numeric.copy()
df_strictly_numeric["is_correct"] = (df_strictly_numeric["numeric_response"] == df_strictly_numeric["correct_answer"]).astype(int)
df_strictly_numeric["iteration"] = df_strictly_numeric.groupby("experiment_id").cumcount() + 1

df_strictly_numeric["emoji"] = df_strictly_numeric["is_correct"].map({1: "✅", 0: "❌"})

# build the wide table (one column per iteration)
hit_table = (
    df_strictly_numeric.pivot(index="experiment_id", columns="iteration", values="emoji")
      .sort_index()
      .sort_index(axis=1)
)

# prepend the summary column "X correct answers of Y total answers"
summary = (
    df_strictly_numeric.groupby("experiment_id")["is_correct"]
      .agg(["sum", "count"])
      .assign(label=lambda s: s.apply(lambda r: f"{r['sum']} correct answers of {r['count']} total answers", axis=1))
)

hit_table["Summary"] = summary.loc[hit_table.index, "label"]

# preview + save
hit_table

iteration,1,2,3,4,5,6,7,8,9,10,11,12,Summary
experiment_id,,,,,,,,,,,,,
1-1,✅,❌,✅,❌,❌,❌,✅,❌,✅,✅,❌,✅,6 correct answers of 12 total answers
1-2,✅,✅,✅,✅,✅,✅,✅,❌,✅,NaN,NaN,NaN,8 correct answers of 9 total answers
1-3,❌,❌,✅,❌,❌,✅,❌,❌,✅,✅,NaN,NaN,4 correct answers of 10 total answers
1-4,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌,NaN,0 correct answers of 11 total answers
1-5,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,12 correct answers of 12 total answers


What's up with 1-4? 

In [97]:
df[df['experiment_id']=='1-4']

,filename,experiment_id,response,tool_chain,numeric_response,all_numbers,correct_answer
30,1-4_19_extracted-info.json,1-4,97,[get_play_metadata],97.0,[97],103
40,1-4_18_extracted-info.json,1-4,95,[get_play_metadata],95.0,[95],103
68,1-4_17_extracted-info.json,1-4,106,[get_play_metadata],106.0,[106],103
77,1-4_16_extracted-info.json,1-4,91,[get_play_metadata],91.0,[91],103
90,1-4_15_extracted-info.json,1-4,101,[get_play_metadata],101.0,[101],103
94,1-4_14_extracted-info.json,1-4,100,[get_play_metadata],100.0,[100],103
102,1-4_12_extracted-info.json,1-4,94,[get_play_metadata],94.0,[94],103
104,1-4_13_extracted-info.json,1-4,91,[get_play_metadata],91.0,[91],103
115,1-4_11_extracted-info.json,1-4,104,[get_play_metadata],104.0,[104],103
122,1-4_20_extracted-info.json,1-4,105,[get_play_metadata],105.0,[105],103
